# Головина Анна Анатольевна

In [321]:
import collections
from nltk.corpus import stopwords
from nltk.tokenize import RegexpTokenizer
import pandas as pd
from pymorphy2 import MorphAnalyzer
from sklearn.metrics import accuracy_score
import spacy
import stanza
from tqdm import tqdm

1. Создание, разметка корпуса и объяснение того, почему этот текст подходит для оценки (какие моменты вы тут считаете трудными для автоматического посттеггинга и почему, в этом вам может помочь второй ридинг). Не забывайте, что разные теггеры могут использовать разные тегсеты: напишите комментарий о том, какой тегсет вы берёте для разметки и почему.

Я выбрала текст какого-то кринжового [стихотворения](https://stihi.ru/2015/03/25/6688) на омоформы т.е. омонимы, у которых совпадают написание и произношение, но различаются грамматические характеристики. Как мне кажется, это полнейший ад для pos-теггеров, поскольку для подобных случаев не подойдет теггинг втупую, нужно учитывать контекст. Помимо того, в тексте встречаются причастия, деепричастия и субстантивированные прилагательные (по типу *нищие*). То есть в тексте присутствуют трудности двух видов: выбор части речи при наличии омонимов (омоформы и субстантивации), а также выбор части речи для слова, совмещающего признаки нескольких частей речи (деепричастие: глагол/глагольная форма или наречие? причастие: глагол/глагольная форма или прилагательное?)

В качестве тегсета выбрала упрощенный список тегов conllu. Теги остаются теми же, кроме тегов CCONJ и SCONJ, потому что многие теггеры используют просто CONJ, и при конвертации придется еще и перепроверять, присвоился тег сочинительного или подчинительного союза, или же прописывать отдельно, какие словоформы относить к CCONJ, а какие к SCONJ, а таких усложнений хочется избежать.

Сначала для подготовки данных к ручной разметке токенизирую текст из файла *text.txt*. Размер корпуса составляет 842 словоформы (знаки препинания просто не учитываются при токенизации, потому что все-таки оцениваем качество pos-теггинга, а не качество разбора знаков препинания).

In [322]:
def tokenization(text):
    """
    Функция для токенизации текстов
    Аргумент - строка с текстом отзыва
    Вывод - список токенов
    """
    text = text.lower()
    tokenizer = RegexpTokenizer(r"[а-я0-9ё]+")
    text_tokenized = tokenizer.tokenize(text)
    return text_tokenized

In [323]:
with open("text.txt", "r", encoding="utf-8") as f:
    text = tokenization(f.read())

Список получившихся словоформ записываю в excel-таблицу в колонку *tokens* для дальнейшей работы вручную. Кажется, что ручная разметка присвоением тегов словоформам прямо в тетрадке - это очень странная затея.

In [324]:
df = pd.DataFrame({"tokens": text})
df.to_excel("annotation.xlsx")

Для частеречного разбора склеиваем список в строку.

In [325]:
text = " ".join(text)

2. Потом вам будет нужно взять три POS теггера для русского языка (udpipe, stanza, natasha, pymorphy, mystem, spacy, deeppavlov) и «прогнать» текст через каждый из них.

Изначально планировала использовать *natasha*, но в данной задаче приличных слов для описания этой бибилиотеки нет, она работает только с именованными сущностями, а значит, игнорирует около 80% текста.
В итоге были выбраны stanza, pymorphy и spacy.

Начнем с stanza. Создаю список кортежей (словоформа, тег). Кортежи использую, поскольку это неизменяемый тип данных, и даже по случайности не получится поменять их составляющие, что позволит до конвертации сохранить данные в исходной форме.

In [326]:
nlp = stanza.Pipeline(lang="ru", processors="tokenize,pos", download_method=None)
doc = nlp(text)

stanza_pos = [
    tuple([str(word.text), str(word.upos)])
    for sent in doc.sentences
    for word in sent.words
]

2023-10-07 23:09:00 INFO: Loading these models for language: ru (Russian):
| Processor | Package          |
--------------------------------
| tokenize  | syntagrus        |
| pos       | syntagrus_charlm |

2023-10-07 23:09:00 INFO: Using device: cpu
2023-10-07 23:09:00 INFO: Loading: tokenize
2023-10-07 23:09:00 INFO: Loading: pos
2023-10-07 23:09:00 INFO: Done loading processors!


Далее spacy. Все то же самое.

In [327]:
ru_nlp = spacy.load("ru_core_news_md")
document = ru_nlp(text)
    
spacy_pos = [tuple([token, str(token.pos_)]) for token in document]

Наконец, pymorphy.

In [328]:
morph = MorphAnalyzer()

pymorphy_pos = [
    tuple([token, str(morph.parse(token)[0].tag.POS)]) for token in tokenization(text)
]

3. Затем оценим accuracy для каждого теггера. Заметьте, что в разных системах имена тегов и части речи могут отличаться, – вам надо будет свести это всё к единому стандарту с помощью какой-то функции-конвертера и сравнить с вашим размеченным руками эталоном - тоже с помощью какого-то кода или функции.

Напишем конвертер тегов в единый формат.

In [329]:
def convert_tags(annotation_row):
    """
    Функция для получения списка тегов
    Аргумент - размеченный корпус (список кортежей типа (словоформа, тег))
    Вывод - список конвертированных тегов
    """
    correct_tags = {
        "ADJ": ["COMP", "ADJF", "ADJS"],
        "ADP": "PREP",
        "ADV": "ADVB",
        "CONJ": ["CCONJ", "SCONJ"],
        "NUM": "NUMR",
        "VERB": ["GRND", "PRED", "PRTS", "PRTF", "INFN"],
        "PART": "PRCL",
        "PRON": "NPRO",
    }

    tags_row = [item[1] for item in annotation_row]
    for i in range(len(tags_row)):
        for tag in correct_tags:
            if tags_row[i] in correct_tags[tag] or tags_row[i] == correct_tags[tag]:
                tags_row[i] = tag
    return tags_row

Далее конвертируем теги для всех теггеров.

In [330]:
stanza_pos = convert_tags(stanza_pos)
spacy_pos = convert_tags(spacy_pos)
pymorphy_pos = convert_tags(pymorphy_pos)

Достаем из таблицы с ручной разметкой столбец тегов и преобразуем его в список.

In [331]:
df_correct = pd.read_excel("correct_annotation.xlsx")
correct = df_correct["POS"].to_list()

Точность stanza составляет ~0.788.

In [332]:
accuracy_score(correct, stanza_pos)

0.7885985748218527

Точность spacy составляет ~0.766.

In [333]:
accuracy_score(correct, spacy_pos)

0.7660332541567696

Точность составляет ~0.786.

In [334]:
accuracy_score(correct, pymorphy_pos)

0.7862232779097387

Таким образом, лучшим теггером является stanza.

4. Дальше вам нужно взять лучший теггер для русского языка и с его помощью написать функцию (chunker), которая выделяет из размеченного текста 3 типа n-грамм, соответствующих какому-то шаблону (к примеру не + какая-то часть речи или NP или сущ.+ наречие и тд). Предложите 3 шаблона (слово + POS-тег / POS-тег + POS-тег) запись которых в словарь, по вашему мнению, улучшила бы качество работы программы из предыдущей домашки. Балл за объяснение того, почему именно эти группы вы взяли, балл за создание такого рода чанкера, балл за за встраивание функции в программу из предыдущей домашки, балл за сравнение качества предсказания тональности с улучшением и без.

Предлагаю следующие шаблоны:
1. не + VERB: чаще всего в негативных отзывах встречаются либо отрицательные императивы наподобие *не связывайтесь*, либо констатации вроде *не воспользуюсь этим сервисом снова*, однако нельзя предугадать, какие конкретно глаголы окажутся под отрицанием, известна лишь предполагаемая часть речи. Такие биграммы можно использовать как плюс к тому, что отзыв отрицательный.
2. очень + (ADJ | ADV): в положительных отзывах часто встречаются положительные наречные или прилагательные харакеристики, подкрепляемые модификатором *очень* (в отрицательных отзывах противоположные единицы). Непонятно, будет ли услуга оказана *очень быстро/долго*, или будет *очень удобный/ужасный* сервис, и какая вообще характеристика будет подчеркиваться, поэтому заменяем тегом и включаем условие "или".
3. NOUN + и + NOUN: очень часто в негативных отзывах ругательства в адрес исполнителей заказа формулируются сочинительной конструкцией с двумя существительными, например *воров и мошенников*, *халтура и безответственность*, *бараны и подонки*. Нельзя угадать, как конкретно обзовут исполнителей, поэтому конъюнкты просто существительные. В положительных же отзывах встречаются сочетание наподобие *безопасность и конфиденциальность*.

Я чуть не померла, пока пыталась придумать, как, например, через \*\*kwargs заставить одну и ту же функцию обрабатывать и биграммы, и триграммы, да еще и разной структуры. В итоге решила сделать просто чанкер под n-граммы с заданным n, и уже вне функции собирать все по шаблонам. Может, я что-то не понимаю, от слова *совсем*, и надо было просто выбирать другие шаблоны, ну да ладно.

In [335]:
def chunker(n, line):
    """
    Функция для деления текста на n-граммы
    Аргументы - порядок n-граммы, текст
    Вывод - список n-грамм
    """
    ngramms = [line[i : i + n] for i in range(len(line) - n + 1)]
    return ngramms

In [357]:
def ngramms(review):
    """
    Функция для вычленения n-грамм из текста через создание размеченного корпуса
    Аргумент - текст
    Вывод - список со списками n-грамм
    """
    nlp = stanza.Pipeline(lang="ru", processors="tokenize,pos", download_method=None)
    doc = nlp(review)

    stanza_review_pos = [
        tuple([str(word.text), str(word.upos)])
        for sent in doc.sentences
        for word in sent.words
    ]

    # первый шаблон:
    ngramms_1 = [
        item
        for item in chunker(2, stanza_review_pos)
        if item[0][0] == "не" and item[1][1] == "VERB"
    ]
    # второй шаблон:
    ngramms_2 = [
        item
        for item in chunker(2, stanza_review_pos)
        if item[0][0] == "очень" and (item[1][1] == "ADJ" or item[1][1] == "ADV")
    ]
    # третий шаблон:
    ngramms_3 = [
        item
        for item in chunker(3, stanza_review_pos)
        if item[0][1] == "NOUN" and item[1][0] == "и" and item[2][1] == "NOUN"
    ]

    all_ngramms = (
        [f"{i[0][0]} {i[1][0]}" for i in ngramms_1]
        + [f"{j[0][0]} {j[1][0]}" for j in ngramms_2]
        + [f"{k[0][0]} {k[1][0]} {k[2][0]}" for k in ngramms_3]
    )

    return all_ngramms

Для удобства и экономии времени (краулер из прошлой домашки фурычит достаточно долго), я записала отзывы в csv-файлы. Хвосты сразу отсеку на тестовый датафрейм, остальное по принципу предыдущей домашки для тренировки.

UPD: для записи в файлы пришлось по новой запускать код предыдущего дз, на сайт записали новые отзывы, поэтому множества положительных и отрицательных слов изменили размерность и состав, поэтому качество модели из предыдущей домашки отличается от качества модели на данный момент. Если в задании было написано встроить функцию в программу из предыдущей домашки, никакие параметры (относительно частотности слов) менять не буду.

In [337]:
df_neg = pd.read_csv("negative_reviews.csv")
df_pos = pd.read_csv("positive_reviews.csv")

test_df = pd.concat([df_neg.iloc[85:], df_pos[85:]], ignore_index=True)

neg_texts = df_neg[:85]["text"].to_list()
pos_texts = df_pos[:85]["text"].to_list()

Теперь просто слегка модифицированный код из предыдущей домашки для оценки качества без улучшения.

In [338]:
morph = MorphAnalyzer()
stop_words = set(stopwords.words("russian"))

In [339]:
def lemmatization(text):
    '''
    Функция для получения лемм слов
    Аргумент - строка с текстом отзыва
    Вывод - список лемм
    '''
    text = text.lower()
    tokenizer = RegexpTokenizer(r"[а-я0-9ё]+")
    text_tokenized = tokenizer.tokenize(text)
    lemmas = [morph.parse(word)[0].normal_form for word in text_tokenized if (word.isalpha() == True and word not in stop_words)]
    return lemmas

In [340]:
neg_lemmas, pos_lemmas = [], []

for i in range(85): # обработаю по 85 первых положительных и отрицательных отзывов
    neg_lemmas += lemmatization(neg_texts[i])
    pos_lemmas += lemmatization(pos_texts[i])

In [341]:
def keywords(list_of_lemmas):
    """
    Функция для получения наиболее частотных слов текста
    Аргумент - список лемм
    Вывод - список отфильтрованных по частотности лемм
    """
    c = collections.Counter()

    for lemma in list_of_lemmas:
        c[lemma] += 1
    # начиная с 6 (т.е. > 5) сильно проседают явно оценочные слова в положительных отзывах
    c_filtered = {i: c[i] for i in c.keys() if c[i] > 4}
    return list(c_filtered.keys())

In [342]:
neg_temp_set = set(keywords(neg_lemmas))
pos_temp_set = set(keywords(pos_lemmas))

In [343]:
neg_set = neg_temp_set.difference(pos_temp_set)
pos_set = pos_temp_set.difference(neg_temp_set)

In [344]:
def review_rating(ratings, review):
    """
    Функция для оценки тональности текста
    Аргумент - текст отзыва
    Вывод - оценка
    """
    review_lemmas = lemmatization(review)

    rev_rating = collections.Counter()
    for grade, word_set in ratings.items():
        word_set = collections.Counter(word_set)
        for lemma in review_lemmas:
            rev_rating[grade] += int(word_set[lemma] > 0)
    return rev_rating.most_common()[0][0]

In [347]:
ratings_dict_sets = {0: neg_set, 1: pos_set}

Оценим качество. Тестовые данные и правильная разметка одинаковы для оценки по множествам, по n-граммам и для совмещенного медота.

In [348]:
test_data = test_df["text"].to_list()
correct_annotation = test_df["sent"].to_list()

In [349]:
predicted_with_sets = [review_rating(ratings_dict_sets, item) for item in test_data]

In [350]:
accuracy_score(correct_annotation, predicted_with_sets)

0.6

Теперь с улучшением. Под улучшением имею в виду добавление функции ngramms, которая включает в себя чанкер и выдает n-граммы. Сначала хочется оценить качество функции без учета множеств положительных и отрицательных слов.

In [374]:
def review_rating_ngramms(ratings, review):
    """
    Функция для оценки тональности текста
    Аргумент - текст отзыва
    Вывод - оценка
    """

    rev_rating = collections.Counter()
    for grade, ngramms in ratings.items():
        ngramms = collections.Counter(ngramms)
        for ngramm in ngramms:
            rev_rating[grade] += int(ngramm in review)
            # проверяем наличие n-граммы, записанной в строковом виде, в принципе в тексте
    return rev_rating.most_common()[0][0]

In [362]:
neg_ngramms = []
pos_ngramms = []

neg_ngramms += [ngramms(item) for item in neg_texts]
pos_ngramms += [ngramms(item) for item in pos_texts]

2023-10-07 23:30:08 INFO: Loading these models for language: ru (Russian):
| Processor | Package          |
--------------------------------
| tokenize  | syntagrus        |
| pos       | syntagrus_charlm |

2023-10-07 23:30:08 INFO: Using device: cpu
2023-10-07 23:30:08 INFO: Loading: tokenize
2023-10-07 23:30:08 INFO: Loading: pos
2023-10-07 23:30:09 INFO: Done loading processors!
2023-10-07 23:30:10 INFO: Loading these models for language: ru (Russian):
| Processor | Package          |
--------------------------------
| tokenize  | syntagrus        |
| pos       | syntagrus_charlm |

2023-10-07 23:30:10 INFO: Using device: cpu
2023-10-07 23:30:10 INFO: Loading: tokenize
2023-10-07 23:30:10 INFO: Loading: pos
2023-10-07 23:30:10 INFO: Done loading processors!
2023-10-07 23:30:11 INFO: Loading these models for language: ru (Russian):
| Processor | Package          |
--------------------------------
| tokenize  | syntagrus        |
| pos       | syntagrus_charlm |

2023-10-07 23:30:11 

Получились списки в списках, так что тут перезапись.

In [366]:
neg_ngramms_ = []
for i in range(len(neg_ngramms)):
    neg_ngramms_ += neg_ngramms[i]
    
pos_ngramms_ = []
for i in range(len(pos_ngramms)):
    pos_ngramms_ += pos_ngramms[i]

Составляем словарь положительных и отрицательных n-грамм.

In [369]:
ratings_dict_ngramms = {0: neg_ngramms_, 1: pos_ngramms_}

Применяем функцию.

In [375]:
predicted_with_ngramms = [review_rating_ngramms(ratings_dict_ngramms, item) for item in test_data]

Оцениваем точность.

In [376]:
accuracy_score(correct_annotation, predicted_with_ngramms)

0.5

Качество только ухудшилось, но есть надежда, что совмещение множеств и n-грамм улучшит качество.

In [385]:
def better_review_rating(ratings_sets, ratings_ngramms, review):
    """
    Функция для оценки тональности текста
    Аргументы - словарь с множествами слов, словарь с n-граммами, текст отзыва
    Вывод - оценка
    """
    review_lemmas = lemmatization(review)

    rev_rating = collections.Counter()
    for grade, word_set in ratings_sets.items():
        word_set = collections.Counter(word_set)
        for lemma in review_lemmas:
            rev_rating[grade] += int(word_set[lemma] > 0)
    for grade, ngramms in ratings_ngramms.items():
        ngramms = collections.Counter(ngramms)
        for ngramm in ngramms:
            rev_rating[grade] += int(ngramm in review)
    return rev_rating.most_common()[0][0]

In [386]:
predicted_mixed = [better_review_rating(ratings_dict_sets, ratings_dict_ngramms, item) for item in test_data]

In [387]:
accuracy_score(correct_annotation, predicted_mixed)

0.6

Ничего не изменилось, короче, лучшее - враг хорошего.

*занавес*